In [2]:
!pip install corner

  Created wheel for corner: filename=corner-2.0.1-cp36-none-any.whl size=11642 sha256=82a4809b0aeb0e148cf17a7f1e5d41afea86cce858fc2e3c93030f5816724183
  Stored in directory: /root/.cache/pip/wheels/70/d8/e5/e0e7974a2a5757483ea5a180c937041cf6872dc9993d78234a
Successfully built corner


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd drive/My\ Drive/Projects/VariationalAE

/content/drive/My Drive/Projects/VariationalAE


In [0]:
import os
import numpy as np
import pandas as pd

In [0]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [0]:
from my_utils import load_data, train_evaluate_model

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
class InfoVAE(nn.Module):
    def __init__(self, nfeat=4, ncode=3, alpha=0, lambd=10000, nhidden=128, nhidden2=35, dropout=0.2):
        super(InfoVAE, self).__init__()
        
        self.ncode = int(ncode)
        self.alpha = float(alpha)
        self.lambd = float(lambd)
        
        self.leaky_relu = nn.LeakyReLU()
        self.encd = nn.Linear(nfeat, nhidden)
        self.d1 = nn.Dropout(p=dropout)
        self.enc2 = nn.Linear(nhidden, nhidden2)
        self.d2 = nn.Dropout(p=dropout)
        self.mu = nn.Linear(nhidden2, ncode)
        self.lv = nn.Linear(nhidden2, ncode)
        
        self.decd = nn.Linear(ncode, nhidden2)
        self.d3 = nn.Dropout(p=dropout)
        self.dec2 = nn.Linear(nhidden2, nhidden)
        self.d4 = nn.Dropout(p=dropout)
        self.outp = nn.Linear(nhidden, nfeat)
        
    def encode(self, x):
        x = self.d1(self.leaky_relu(self.encd(x)))
        x = self.d2(self.leaky_relu(self.enc2(x)))
        mu = self.mu(x)
        logvar = self.lv(x)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)
    
    def decode(self, x):
        x = self.d3(self.leaky_relu(self.decd(x)))
        x = self.d4(self.leaky_relu(self.dec2(x)))
        x = self.outp(x)
        return x
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        mu = mu.to(device)
        logvar = logvar.to(device)
        z = self.reparameterize(mu, logvar).to(device)
        return self.decode(z), mu, logvar
    
    # https://ermongroup.github.io/blog/a-tutorial-on-mmd-variational-autoencoders/
    def compute_kernel(self, x, y):
        x_size = x.size(0)
        y_size = y.size(0)
        dim = x.size(1)
        x = x.unsqueeze(1).to(device) # (x_size, 1, dim)
        y = y.unsqueeze(0).to(device) # (1, y_size, dim)
        tiled_x = x.expand(x_size, y_size, dim).to(device)
        tiled_y = y.expand(x_size, y_size, dim).to(device)
        # The example code divides by (dim) here, making <kernel_input> ~ 1/dim
        # excluding (dim) makes <kernel_input> ~ 1
        kernel_input = (tiled_x - tiled_y).pow(2).mean(2)#/float(dim)
        return torch.exp(-kernel_input) # (x_size, y_size)
    
    # https://ermongroup.github.io/blog/a-tutorial-on-mmd-variational-autoencoders/
    def compute_mmd(self, x, y):
        xx_kernel = self.compute_kernel(x,x)
        yy_kernel = self.compute_kernel(y,y)
        xy_kernel = self.compute_kernel(x,y)
        return torch.mean(xx_kernel) + torch.mean(yy_kernel) - 2*torch.mean(xy_kernel)
    
    def loss(self, x):
        recon_x, mu, logvar = self.forward(x)
        MSE = torch.sum(0.5 * (x - recon_x).pow(2))
        
        # KL divergence (Kingma and Welling, https://arxiv.org/abs/1312.6114, Appendix B)
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        #return MSE + self.beta*KLD, MSE
                
        # https://ermongroup.github.io/blog/a-tutorial-on-mmd-variational-autoencoders/
        true_samples = Variable(torch.randn(200, self.ncode), requires_grad=False)
        z = self.reparameterize(mu, logvar) #duplicate call
        # compute MMD ~ 1, so upweight to match KLD which is ~ n_batch x n_code
        MMD = self.compute_mmd(true_samples,z) * x.size(0) * self.ncode
        return MSE + (1-self.alpha)*KLD + (self.lambd+self.alpha-1)*MMD, MSE, KLD, MMD

In [0]:
tag = '4D/'
#os.makedirs(tag)

In [0]:
train_dl, valid_dl, train_x, train_y, test_x, test_y = load_data(batch_size = 512)

In [0]:
def train(): #model, optimizer, epoch, min_valid_loss, badepochs
    model.train()
    train_loss = 0
    train_logL = 0
    for batch_idx, data in enumerate(train_dl):
        x = data[0].to(device)
        optimizer.zero_grad()
        loss, logL, KLD, MMD = model.loss(x)
        loss = loss.to(device)
        logL = logL.to(device)
        KLD = KLD.to(device)
        MMD = MMD.to(device)
        loss.backward()
        train_loss += loss.item()
        train_logL += logL.item()
        optimizer.step()
    train_loss /= len(train_dl.dataset)
    
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        valid_logL = 0
        valid_KLD = 0
        valid_MMD = 0

        for data in valid_dl:
            x = data[0].to(device)
            loss, logL, KLD, MMD = model.loss(x)
            valid_loss += loss.item()
            valid_logL += logL.item()
            valid_KLD += KLD.item()
            valid_MMD += MMD.item()
        
        valid_loss /= len(valid_dl.dataset)
        valid_logL /= len(valid_dl.dataset)
        valid_KLD  /= len(valid_dl.dataset)
        valid_MMD  /= len(valid_dl.dataset)
    return valid_loss, valid_logL, valid_KLD, valid_MMD

In [0]:
class EarlyStopper: #Stops the run if loss starts to diverge
    def __init__(self, precision=1e-3, patience=10):
        self.precision = precision
        self.patience = patience
        self.badepochs = 0
        self.min_valid_loss = float('inf')
        
    def step(self, valid_loss):
        if valid_loss < self.min_valid_loss*(1-self.precision):
            self.badepochs = 0
            self.min_valid_loss = valid_loss
        else:
            self.badepochs += 1
        return not (self.badepochs == self.patience)

In [0]:
epochs = 100
log_interval = 10
mdl_ncode = 3
n_config = 1000
nfeat = 4

In [0]:
mdl_MSE = np.zeros(n_config)
mdl_KLD = np.zeros(n_config)
mdl_MMD = np.zeros(n_config)

In [0]:
from tqdm import tqdm
from tqdm import tqdm_notebook

In [0]:
for config in range(n_config):
    alpha = 0
    lambd = np.exp(np.random.uniform(0, np.log(1e5)))
    dropout = 0#0.9*np.random.uniform()
    dfac = 1./(1.-dropout)

    nhidden = 200
    nhidden2 = 200
    print('config %i, alpha = %0.1f, lambda = %0.1f, dropout = %0.2f; 2 hidden layers with %i, %i nodes' % (config, alpha, lambd, dropout, nhidden, nhidden2))
    model = InfoVAE(alpha=alpha, lambd=lambd, nfeat=nfeat, nhidden=nhidden, nhidden2=nhidden2, ncode=mdl_ncode, dropout=dropout)
    model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', verbose=True, patience=5)
    stopper = EarlyStopper(patience=10)

    for epoch in tqdm_notebook((range(1, epochs + 1))):
        valid_loss, valid_logL, valid_KLD, valid_MMD = train()
        if epoch % log_interval == 0:
            print('====> Epoch: {} VALIDATION Loss: {:.2e} logL: {:.2e} KL: {:.2e} MMD: {:.2e}'.format(
                  epoch, valid_loss, valid_logL, valid_KLD, valid_MMD))
        
        optimizer.step()
        scheduler.step(valid_loss)
        if (not stopper.step(valid_loss)) or (epoch == epochs):
            print('Stopping')
            print('====> Epoch: {} VALIDATION Loss: {:.2e} logL: {:.2e} KL: {:.2e} MMD: {:.2e}'.format(
                  epoch, valid_loss, valid_logL, valid_KLD, valid_MMD))
            model.MSE = -valid_logL
            model.KLD = valid_KLD
            model.MMD = valid_MMD
            mdl_MSE[config] = model.MSE
            mdl_KLD[config] = model.KLD
            mdl_MMD[config] = model.MMD
            torch.save(model, tag+'/%04i.pth' % config)
            break

config 0, alpha = 0.0, lambda = 226.4, dropout = 0.00; 2 hidden layers with 200, 200 nodes


====> Epoch: 10 VALIDATION Loss: 5.33e+00 logL: 1.96e+00 KL: 2.49e-02 MMD: 1.48e-02
Epoch    11: reducing learning rate of group 0 to 1.0000e-05.
Epoch    19: reducing learning rate of group 0 to 1.0000e-06.
====> Epoch: 20 VALIDATION Loss: 5.52e+00 logL: 1.95e+00 KL: 2.65e-02 MMD: 1.58e-02
Stopping
====> Epoch: 23 VALIDATION Loss: 5.57e+00 logL: 1.95e+00 KL: 2.65e-02 MMD: 1.59e-02
config 1, alpha = 0.0, lambda = 7047.6, dropout = 0.00; 2 hidden layers with 200, 200 nodes


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type InfoVAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


====> Epoch: 10 VALIDATION Loss: 1.11e+02 logL: 2.00e+00 KL: 8.92e-03 MMD: 1.55e-02
Epoch    14: reducing learning rate of group 0 to 1.0000e-05.
Stopping
====> Epoch: 18 VALIDATION Loss: 1.15e+02 logL: 2.00e+00 KL: 4.89e-03 MMD: 1.60e-02
config 2, alpha = 0.0, lambda = 158.7, dropout = 0.00; 2 hidden layers with 200, 200 nodes


====> Epoch: 10 VALIDATION Loss: 4.21e+00 logL: 1.73e+00 KL: 1.29e-01 MMD: 1.49e-02
Epoch    10: reducing learning rate of group 0 to 1.0000e-05.
Epoch    17: reducing learning rate of group 0 to 1.0000e-06.
====> Epoch: 20 VALIDATION Loss: 4.52e+00 logL: 1.70e+00 KL: 1.45e-01 MMD: 1.69e-02
Stopping
====> Epoch: 21 VALIDATION Loss: 4.23e+00 logL: 1.71e+00 KL: 1.45e-01 MMD: 1.51e-02
config 3, alpha = 0.0, lambda = 18.6, dropout = 0.00; 2 hidden layers with 200, 200 nodes


...

====> Epoch: 10 VALIDATION Loss: 2.18e+00 logL: 1.42e+00 KL: 3.53e-01 MMD: 1.67e-02
====> Epoch: 20 VALIDATION Loss: 2.13e+00 logL: 1.36e+00 KL: 4.08e-01 MMD: 1.52e-02
Epoch    20: reducing learning rate of group 0 to 1.0000e-05.
====> Epoch: 30 VALIDATION Loss: 2.15e+00 logL: 1.36e+00 KL: 4.06e-01 MMD: 1.60e-02
Epoch    34: reducing learning rate of group 0 to 1.0000e-06.
====> Epoch: 40 VALIDATION Loss: 2.10e+00 logL: 1.35e+00 KL: 4.07e-01 MMD: 1.42e-02
Epoch    43: reducing learning rate of group 0 to 1.0000e-07.
Stopping
====> Epoch: 47 VALIDATION Loss: 2.15e+00 logL: 1.34e+00 KL: 4.07e-01 MMD: 1.65e-02
config 47, alpha = 0.0, lambda = 96091.7, dropout = 0.00; 2 hidden layers with 200, 200 nodes


Epoch     7: reducing learning rate of group 0 to 1.0000e-05.
====> Epoch: 10 VALIDATION Loss: 1.36e+03 logL: 1.99e+00 KL: 1.10e-02 MMD: 1.42e-02
Stopping
====> Epoch: 11 VALIDATION Loss: 1.46e+03 logL: 1.99e+00 KL: 1.18e-02 MMD: 1.52e-02
config 48, alpha = 0.0, lambda = 12.6, dropout = 0.00; 2 hidden layers with 200, 200 nodes


====> Epoch: 10 VALIDATION Loss: 1.96e+00 logL: 1.37e+00 KL: 3.86e-01 MMD: 1.75e-02
Epoch    18: reducing learning rate of group 0 to 1.0000e-05.
====> Epoch: 20 VALIDATION Loss: 1.92e+00 logL: 1.33e+00 KL: 4.13e-01 MMD: 1.53e-02
Epoch    26: reducing learning rate of group 0 to 1.0000e-06.
====> Epoch: 30 VALIDATION Loss: 1.93e+00 logL: 1.33e+00 KL: 4.16e-01 MMD: 1.54e-02
Stopping
====> Epoch: 30 VALIDATION Loss: 1.93e+00 logL: 1.33e+00 KL: 4.16e-01 MMD: 1.54e-02
config 49, alpha = 0.0, lambda = 4.6, dropout = 0.00; 2 hidden layers with 200, 200 nodes


====> Epoch: 10 VALIDATION Loss: 1.81e+00 logL: 1.34e+00 KL: 4.09e-01 MMD: 1.65e-02
====> Epoch: 20 VALIDATION Loss: 1.81e+00 logL: 1.33e+00 KL: 4.15e-01 MMD: 1.69e-02
Epoch    21: reducing learning rate of group 0 to 1.0000e-05.
Stopping
====> Epoch: 21 VALIDATION Loss: 1.81e+00 logL: 1.34e+00 KL: 4.16e-01 MMD: 1.54e-02
config 50, alpha = 0.0, lambda = 3.5, dropout = 0.00; 2 hidden layers with 200, 200 nodes


====> Epoch: 10 VALIDATION Loss: 1.80e+00 logL: 1.34e+00 KL: 4.10e-01 MMD: 1.68e-02
